In [3]:
import os
import pcapy as p
from scapy.all import *
import matplotlib.pyplot as plt

import pandas as pd

import struct
import numpy as np

In /home/kelu/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/kelu/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/kelu/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/kelu/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/kelu/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_c

In [4]:
data = "66Mhz_B.pcapng"

In [19]:
packets = rdpcap(data)
Config = []
IMGS = []
IMGSsmol = []
idxImags = []
IMGS2 = []
IMGSsmol2 = []
idxImags2 = []
start = 0
ix = 1
for k in packets:
    k = k[Raw].load
    if k[8] == 0x53: # URB submit
        if k[9] == 0x02: # URB submit
            TYPE = "URB_CONTROL"
        elif k[9] == 0x03: # BULK
            TYPE = "BULK"
        else:
            TYPE = "OTHER"
        EP = k[10] # EP
        if EP == 0:
            SENS ="OUT"
        elif EP ==130:
            SENS = "IN"
        elif EP==1:
            SENS = "OUT"
        else:
            SENS = "IN"
            
        if TYPE == "BULK":
            sizeRequested = k[33]*256+k[32] 
            DIC = {'ID': ix,"Sens":SENS,"TYPE":TYPE,"EP":k[10],"sizeRequested":sizeRequested,"LEN":len(k)}
        else:
            bmRequestType = k[40]
            bmRequest = k[41]
            bmValue  =  256*k[43]+k[42] 
            bmIndex  = k[44]+256*k[45]  
            bmLength =  k[46]+256*k[47]  

            #print(ix,hex(bmRequestType),bmRequest,hex(bmValue),bmIndex,bmLength,TYPE,SENS,hex(EP))
            DIC = {'ID': ix,"Sens":SENS,"TYPE":TYPE,"EP":k[10],"bmRequestType":bmRequestType,
                           "bmRequest":bmRequest,"bmValue":bmValue,"bmIndex":bmIndex,"bmLength":bmLength,"LEN":len(k)
                   }
        if len(k) > 64:
            DIC["payload"] = k[64:]
        Config.append(DIC)
            
    if k[8] == 0x43: # URB receive
        if 0:
            if (len(k) == 65):
                print(ix,hex(k[-1]))
            if (len(k) == 66):
                print(ix,hex(k[-1]+256*k[-2]  ))
                
    if (k[8] == 0x43) & ((k[10] == 0x86)): # URB receive  
        if len(k) > 20000:
            IMGS.append(k[64:])
            #print(ix)
            idxImags.append(str(ix)+"full")
        else:
            IMGSsmol.append(k[64:])
            idxImags.append(str(ix)+"short")
    if (k[8] == 0x43) & ((k[10] == 0x82)): # URB receive  
        if len(k) > 20000:
            IMGS2.append(k[64:])
            #print(ix)
            idxImags2.append(str(ix)+"full")
        else:
            IMGSsmol2.append(k[64:])
            idxImags2.append(str(ix)+"short")   
    ix +=1

In [20]:
df = pd.DataFrame(Config) 
DF = df.drop_duplicates(subset=["bmRequestType","bmRequest","bmValue","bmIndex","bmLength","sizeRequested","payload"])#.sort_values(by=["bmRequest","bmValue"])
print( len(df),len(DF) )
DF

195 13


,ID,Sens,TYPE,EP,bmRequestType,bmRequest,bmValue,bmIndex,bmLength,LEN,payload,sizeRequested
0,1,IN,URB_CONTROL,128,128.0,6.0,768.0,0.0,255.0,64,NaN,NaN
1,3,IN,URB_CONTROL,128,128.0,6.0,771.0,1033.0,255.0,64,NaN,NaN
3,7,IN,URB_CONTROL,128,128.0,6.0,770.0,1033.0,255.0,64,NaN,NaN
4,9,IN,URB_CONTROL,128,192.0,241.0,0.0,0.0,4.0,64,NaN,NaN
5,11,IN,URB_CONTROL,128,192.0,3.0,1.0,33792.0,4.0,64,NaN,NaN
6,13,IN,URB_CONTROL,128,192.0,3.0,1.0,32768.0,4.0,64,NaN,NaN
7,15,OUT,URB_CONTROL,0,64.0,3.0,1.0,32768.0,4.0,68,b'\x00\x02!\x01',NaN
8,17,IN,URB_CONTROL,128,192.0,207.0,1.0,0.0,152.0,64,NaN,NaN
9,19,OUT,BULK,1,NaN,NaN,NaN,NaN,NaN,84,b'\x00\x00\x00\x00\x02\x02\x00\x00\x00\x00\x00...,20.0
10,21,OUT,BULK,1,NaN,NaN,NaN,NaN,NaN,84,b'\x00\x00\x00\x00\x82\x03\x00\x00\x00\x00\x00...,20.0


In [24]:
DF.to_pickle("conf.pkl")

In [40]:
#USB
import usb.core
import usb.util 
# Figures
import pandas as pd
import numpy as np
# Utilities
import json
import base64
import struct

In [41]:
device = usb.core.find(idVendor=0x0403, idProduct=0x601e)
    # initialize device
#usb.util.dispose_resources(device)
device.reset()
# find first USBTMC interface
IFS = []
for cfg in device:
    for iface in cfg:
        #print(iface)
        IFS.append(iface)

device.set_configuration(cfg)

# claim interface
for iface in IFS:
    print(" IFACE-> ")
    usb.util.claim_interface(device, iface)

    # don't need to set altsetting - USBTMC devices have 1 altsetting as per the spec

    EP = []
    for ep in iface:
        ep_dir = usb.util.endpoint_direction(ep.bEndpointAddress)
        ep_type = usb.util.endpoint_type(ep.bmAttributes)
        EP.append(ep)
        print(ep)

 IFACE-> 
      ENDPOINT 0x1: Bulk OUT ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :    0x1 OUT
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :  0x400 (1024 bytes)
       bInterval        :    0x0
      ENDPOINT 0x81: Interrupt IN ==========================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :   0x81 IN
       bmAttributes     :    0x3 Interrupt
       wMaxPacketSize   :   0x40 (64 bytes)
       bInterval        :    0x9
 IFACE-> 
      ENDPOINT 0x2: Bulk OUT ===============================
       bLength          :    0x7 (7 bytes)
       bDescriptorType  :    0x5 Endpoint
       bEndpointAddress :    0x2 OUT
       bmAttributes     :    0x2 Bulk
       wMaxPacketSize   :  0x400 (1024 bytes)
       bInterval        :    0x0
      ENDPOINT 0x82: Bulk IN ===============================
       bLength   

In [33]:
print(EP)

[<ENDPOINT 0x2: Bulk OUT>, <ENDPOINT 0x82: Bulk IN>]


In [42]:
for index, row in DF.iterrows():
    if  row["EP"] == 0:
        print(row["ID"])
        bmRT = int(row["bmRequestType"])
        bmR = int(row["bmRequest"])
        wV = int(row["bmValue"])
        wI = int(row["bmIndex"])
        if row["payload"] == np.nan:
            PL = row["payload"]
        else:
            PL = int(row["bmLength"])
        print(bmRT,bmR,wV,wI,PL)
        device.ctrl_transfer(bmRequestType= bmRT,bRequest= bmR, wValue= wV, wIndex= wI , data_or_wLength=  PL)
        print("IK")
print("Done")




5
35 1 16 1 0


USBError: [Errno 19] No such device (it may have been disconnected)

In [ ]:
data2 = [] # ENDPOINT 0x82: Bulk IN ===============================
i = 0
while i < 4*2048*120//512: # 2 because we're reading 2bytes words
    data.append(EP[0].read(0x200))
    data2.append(EP[1].read(0x200)) 
    i += 1